# Detección de anomalías

En este ejercicio vamos a trabajar en detectar anomalías. Para ello vamos a probar distintas técnicas como la transformación inversa de modelos de reducción de dimensionalidad o Isolation Forest.



In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.decomposition import SparsePCA
from sklearn.decomposition import KernelPCA

from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

### Los datos

Vamos a trabajar con el dataset creditcard con el que trabajamos en el examen y en la entrega de los autoencoders. Como en los casos anteriores, lo primero que haremos será cargar el dataset guardar en una variable llamada X los atributos y en una variable Y las clases resultantes y lo vamos a escalar entre 0 y 1 los atributos.

In [2]:
file_name = 'examen-parcial-creditcard.csv'
df=pd.read_csv(file_name)
df=df.drop(columns=['Time'])
X=df.copy().drop(columns=['Class'])
Y=df['Class'].copy()
scl=MinMaxScaler()
X= pd.DataFrame(scl.fit_transform(X),columns=X.columns)

A continuación dividismos los sets  X e y cada uno de ellos en dos subsets, uno de entrenamiento y otro de validación. El de entrenamiento debería tener el 90% de los puntos y debería estar estratificado en función de los valores de Y.

In [3]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,
                                                 test_size=0.1,
                                                 random_state=42,
                                                 shuffle=True,
                                                 stratify=Y)

### Detección de anomalías usando PCA.

Aparte de las metodologías ya testadas en el curso en referencia a clustering (distancia a los centroides en KMEANS y la búsqueda de los puntos considerados como ruido en DBSCAN) y a reducción de dimensionalidades (como la aplicación de la tercera entrega (SOM) y cuarta (autoencoders)), vamos a probar alguna técnica adicional. La primera será usando PCA. Probaremos a reducir la dimensionalidad y, posteriormente, la reconstrucción, etiquetando como sospechosos aquellos que presenten un error de reconstrucción mayor. 

Para ello vamos a hacer una reducción de dimensionalidades hasta 10 dimensiones desarrollando las PCAs con el set de entrenamiento. A continuación se transformará el set de test y se deshará la transformación. 

In [4]:
pca =  PCA(n_components=10).fit(x_train)
x_test_pca = pca.transform(x_test)
x_test_pca_inverse = pca.inverse_transform(x_test_pca)

Una vez deshecha la transformación, calcule el error de reconstrucción como la diferencia euclídea entre el vector orginal y el vector reconstruido. Guarde en un dataframe el error para cada vector de entrada y añada una columna con la verdadera clase del movimiento (si es fraudulento o no).

In [5]:
diffs = x_test - x_test_pca_inverse
df = pd.DataFrame(diffs)
df.insert(0, 'Class', y_test, True)
df['error'] = np.linalg.norm(diffs, axis=1)
df['error'].describe()

count    28481.000000
mean         0.077876
std          0.046927
min          0.022711
25%          0.056153
50%          0.069006
75%          0.085668
max          1.104067
Name: error, dtype: float64

Probad con distintos valores de componentes principales. Y evaluad los errores de reconstrucción para diferente número de componentes principales.

In [6]:
for n_components in range(10, 30, 5):
    pca =  PCA(n_components=n_components).fit(x_train)
    x_test_pca = pca.transform(x_test)
    x_test_pca_inverse = pca.inverse_transform(x_test_pca)

    diffs = x_test - x_test_pca_inverse
    df = pd.DataFrame(diffs)
    df.insert(0, 'Class', y_test, True)
    df['error'] = np.linalg.norm(diffs, axis=1)
    print(f"Error by class for {n_components} components: ")
    print(df.groupby('Class')['error'].mean())

Error by class for 10 components: 
Class
0    0.077163
1    0.491996
Name: error, dtype: float64
Error by class for 15 components: 
Class
0    0.043086
1    0.345963
Name: error, dtype: float64
Error by class for 20 components: 
Class
0    0.016815
1    0.072929
Name: error, dtype: float64
Error by class for 25 components: 
Class
0    0.007212
1    0.045444
Name: error, dtype: float64


# Sparse PCA
Realice un proceso similar pero con un PCA dispersos con alpha 0.0001 y 10 componentes. Desarrolle el modelo de PCA disperso con los datos de entrenamiento, transforme y deshaga la transformación del set de test y evalúe el error de reconstrucción para los datos de los movimientos legales y fraudulentos.

In [7]:
n_components = 10
alpha = 0.0001
random_state = 42
n_jobs = -1

sparse_pca = SparsePCA(n_components=n_components, alpha=alpha, random_state=random_state, n_jobs=n_jobs).fit(x_train)

x_test_sparse_pca = sparse_pca.transform(x_test)
x_test_sparse_pca_inverse = sparse_pca.inverse_transform(x_test_sparse_pca)

diffs = x_test - x_test_sparse_pca_inverse
df = pd.DataFrame(diffs)
df.insert(0, 'Class', y_test, True)
df['error'] = np.linalg.norm(diffs, axis=1)
df['error'].describe()

count    28481.000000
mean         0.077935
std          0.046854
min          0.023289
25%          0.056230
50%          0.069088
75%          0.085721
max          1.102811
Name: error, dtype: float64

Probad diferentes valores de alpha y del número de componentes

In [8]:
for alpha in range(10):
    alpha = 0.0001*10**alpha
    sparse_pca = SparsePCA(n_components=n_components, alpha=alpha, random_state=random_state, n_jobs=n_jobs).fit(x_train)

    x_test_sparse_pca = sparse_pca.transform(x_test)
    x_test_sparse_pca_inverse = sparse_pca.inverse_transform(x_test_sparse_pca)

    diffs = x_test - x_test_sparse_pca_inverse
    df = pd.DataFrame(diffs)
    df.insert(0, 'Class', y_test, True)
    df['error'] = np.linalg.norm(diffs, axis=1)
    print(f"Error by class for {alpha} alpha: ")
    print(df.groupby('Class')['error'].mean())

Error by class for 0.0001 alpha: 
Class
0    0.077224
1    0.490444
Name: error, dtype: float64
Error by class for 0.001 alpha: 
Class
0    0.077224
1    0.490444
Name: error, dtype: float64
Error by class for 0.01 alpha: 
Class
0    0.077234
1    0.489888
Name: error, dtype: float64
Error by class for 0.1 alpha: 
Class
0    0.077281
1    0.487665
Name: error, dtype: float64
Error by class for 1.0 alpha: 
Class
0    0.077282
1    0.487666
Name: error, dtype: float64
Error by class for 10.0 alpha: 
Class
0    0.077282
1    0.487666
Name: error, dtype: float64
Error by class for 100.0 alpha: 
Class
0    0.210256
1    0.743746
Name: error, dtype: float64
Error by class for 1000.0 alpha: 
Class
0    0.210256
1    0.743746
Name: error, dtype: float64
Error by class for 10000.0 alpha: 
Class
0    0.210256
1    0.743746
Name: error, dtype: float64
Error by class for 100000.0 alpha: 
Class
0    0.210256
1    0.743746
Name: error, dtype: float64


# Kernel PCA

Realice un proceso similar con nu PCA kernelizado con un kernel de base radial y 10 componentes. Evalúe de la misma manera los errores de reconstrucción de los movimientos legales y los movimientos fraudulentos y evalúe si resulta útil para marcar movimientos como fraudulentos. 

In [14]:
n_components = 10
kernel = 'rbf'
gamma = None
random_state = 42

kernel_pca = KernelPCA(n_components=n_components, kernel=kernel, gamma=gamma, random_state=random_state, fit_inverse_transform=True)
kernel_pca.fit(x_train[:2000])

x_test_kernel_pca = kernel_pca.transform(x_test)
x_test_kernel_pca_inverse = kernel_pca.inverse_transform(x_test_kernel_pca)

diffs = x_test - x_test_kernel_pca_inverse
df = pd.DataFrame(diffs)
df.insert(0, 'Class', y_test, True)
df['error'] = np.linalg.norm(diffs, axis=1)
print(df.groupby('Class')['error'].mean())

Class
0    0.202546
1    0.724815
Name: error, dtype: float64


# Isolation Forest

A continuación use Isolation Forest para deteccióni de anomalías. Use 1000 árboles con un 0.5% de contaminación. Evalúe el score que le predice a los movimientos fraudulentos y a los legales. 

In [12]:
n_estimators = 1000
max_samples = 'auto'
contamination = 0.005
random_state = 42

iso_for = IsolationForest(
    n_estimators=n_estimators,
    max_samples=max_samples,
    contamination=contamination,
    random_state=random_state
).fit(x_train)

y_test_iso_for = iso_for.predict(x_test)

df = pd.DataFrame(x_test)
df.insert(0, 'Class', y_test, True)
df['iso_for'] = y_test_iso_for
df['iso_for'] = df['iso_for'].map({1: 0, -1: 1})
df.groupby('Class')['iso_for'].mean()

Class
0    0.004854
1    0.510204
Name: iso_for, dtype: float64

# Local Outlier Factor

Por último pruebe con LOF con 20 vecinos y un 0.5% de contaminación.

In [13]:
n_neighbors = 20
contamination = 0.005

LOF = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination)
LOF.fit(x_train)

y_test_LOF = LOF.fit_predict(x_test)

df = pd.DataFrame(x_test)
df.insert(0, 'Class', y_test, True)
df['LOF'] = y_test_LOF
df['LOF'] = df['LOF'].map({1: 0, -1: 1})
df.groupby('Class')['LOF'].mean()

Class
0    0.004854
1    0.102041
Name: LOF, dtype: float64